# FIKH Model: Small Amplitude Oscillatory Shear (SAOS)

## Protocol-Specific Context

**SAOS** reveals **fractional viscoelasticity** in the frequency domain via:

1. **Fractional Maxwell modulus**: $G^*(\omega) = G_0 \frac{(i\omega\tau)^{\alpha}}{1 + (i\omega\tau)^{\alpha}}$
2. **Cole-Cole depression**: Depressed semicircle with angle $\theta = (1-\alpha)\pi/2$
3. **Power-law regimes**: $G' \sim \omega^{\alpha}$ (low freq), $G'' \sim \omega^{\alpha}$ (terminal)

**Why this matters**: SAOS provides a **direct map from $\alpha$ to measurable quantities**. The Cole-Cole depression angle is $\theta = (1-\alpha)\pi/2$, so $\alpha = 0.5$ gives exactly 45° depression — a clear visual signature.

> **Physical insight**: Fractional viscoelasticity broadens the relaxation spectrum. Classical Maxwell ($\alpha=1$) has one time scale; FIKH with $\alpha < 1$ has a continuous distribution of relaxation times following power-law $\tau^{-(1-\alpha)}$.

> **Handbook:** See [FIKH SAOS Protocol](../../docs/source/models/fikh/fikh.rst#saos-small-amplitude-oscillatory) for fractional Maxwell derivation and Cole-Cole analysis.

## Learning Objectives

1. Compute FIKH SAOS moduli ($G'$, $G''$) from calibrated parameters
2. Observe Cole-Cole depression and measure angle $\theta$
3. Compare fractional Maxwell ($\alpha < 1$) vs classical ($\alpha=1$) frequency response
4. Extract $\alpha$ from Cole-Cole plot or $\tan(\delta)$ slope
5. Validate NLSQ + Bayesian inference on frequency-domain data

## Prerequisites

- NB01: Flow curve (parameter calibration)
- Basic linear viscoelasticity (storage/loss modulus)

**Estimated Time:** 3-5 minutes (fast), 12-18 minutes (full)

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

# Robust path resolution for execution from any directory
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FIKH

_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    compute_fit_quality,
    generate_synthetic_saos,
    get_fikh_param_names,
    load_fikh_parameters,
    print_alpha_interpretation,
    print_convergence_summary,
    print_parameter_comparison,
    save_fikh_results,
    set_model_parameters,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

# FAST_MODE: controlled by environment variable
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

jax, jnp = safe_import_jax()
verify_float64()

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

## 2. Theory: Fractional SAOS Response

In small-amplitude oscillatory shear, the complex modulus $G^* = G' + iG''$ characterizes linear viscoelastic behavior.

### Classical Maxwell Element
$$
G'(\omega) = G \frac{(\omega\tau)^2}{1 + (\omega\tau)^2}, \quad
G''(\omega) = G \frac{\omega\tau}{1 + (\omega\tau)^2}
$$

### Fractional Maxwell Element
The fractional order $\alpha$ modifies the frequency dependence:
- **Low frequency**: $G' \sim \omega^{2\alpha}$, $G'' \sim \omega^\alpha$
- **High frequency**: Approaches plateau modulus

### Physical Interpretation
- **Lower alpha**: Broader relaxation spectrum, gradual transition
- **Higher alpha**: Sharper crossover (classical Maxwell behavior)

## 3. Load Calibrated Parameters

In [ ]:
# Try to load parameters from NB01
try:
    calibrated_params = load_fikh_parameters("fikh", "flow_curve")
    print("Loaded calibrated parameters from NB01:")
    for name, val in calibrated_params.items():
        print(f"  {name:15s} = {val:.4g}")
except FileNotFoundError:
    print("NB01 parameters not found. Using defaults.")
    calibrated_params = None

In [ ]:
# Create model and set parameters
model = FIKH(include_thermal=False, alpha_structure=0.7)

if calibrated_params is not None:
    set_model_parameters(model, calibrated_params)

param_names = get_fikh_param_names(include_thermal=False)
print(f"\nModel parameters:")
for name in param_names:
    print(f"  {name:15s} = {model.parameters.get_value(name):.4g}")

## 4. Generate Synthetic SAOS Data

In [ ]:
# Generate synthetic SAOS data with 3% noise
# Use fewer frequency points for faster execution
OMEGA_RANGE = (0.1, 10.0)  # Narrower range
N_POINTS = 15  # Fewer points for faster fitting
NOISE_LEVEL = 0.03

omega_data, G_prime_data, G_double_prime_data = generate_synthetic_saos(
    model,
    omega_range=OMEGA_RANGE,
    n_points=N_POINTS,
    noise_level=NOISE_LEVEL,
    seed=42,
)

print(f"Generated synthetic SAOS data:")
print(f"  Frequency range: [{omega_data.min():.4f}, {omega_data.max():.2f}] rad/s")
print(f"  G' range: [{G_prime_data.min():.4g}, {G_prime_data.max():.4g}] Pa")
print(f"  G'' range: [{G_double_prime_data.min():.4g}, {G_double_prime_data.max():.4g}] Pa")
print(f"  Noise level: {NOISE_LEVEL*100:.0f}%")

In [ ]:
# Plot synthetic SAOS data
fig, ax = plt.subplots(figsize=(10, 6))

ax.loglog(omega_data, G_prime_data, "ko", markersize=7, label="G' (Storage)")
ax.loglog(omega_data, G_double_prime_data, "s", color="gray", markersize=6, label="G'' (Loss)")

ax.set_xlabel("Frequency ω [rad/s]", fontsize=12)
ax.set_ylabel("Modulus [Pa]", fontsize=12)
ax.set_title("Synthetic SAOS Data (FIKH Model)", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Alpha Effect on SAOS

In [ ]:
# Compare SAOS response for different alpha values
alpha_values = [0.3, 0.5, 0.7, 0.9, 0.99]
omega_fine = np.logspace(np.log10(OMEGA_RANGE[0]), np.log10(OMEGA_RANGE[1]), 100)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(alpha_values)))

original_alpha = model.parameters.get_value("alpha_structure")

for alpha, color in zip(alpha_values, colors):
    model.parameters.set_value("alpha_structure", alpha)
    
    G_star = model.predict_oscillation(omega_fine, gamma_0=0.001, n_cycles=5)
    G_star = np.asarray(G_star)
    G_p = np.real(G_star)
    G_pp = np.imag(G_star)
    
    ax1.loglog(omega_fine, np.maximum(G_p, 1e-10), "-", color=color, lw=2, label=f"α = {alpha:.2f}")
    ax2.loglog(omega_fine, np.maximum(G_pp, 1e-10), "-", color=color, lw=2, label=f"α = {alpha:.2f}")

model.parameters.set_value("alpha_structure", original_alpha)

# Add data points
ax1.loglog(omega_data, G_prime_data, "ko", markersize=5, alpha=0.5, label="Data")
ax2.loglog(omega_data, G_double_prime_data, "ko", markersize=5, alpha=0.5, label="Data")

ax1.set_xlabel("Frequency [rad/s]", fontsize=12)
ax1.set_ylabel("G' [Pa]", fontsize=12)
ax1.set_title("Storage Modulus vs Alpha", fontsize=13)
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

ax2.set_xlabel("Frequency [rad/s]", fontsize=12)
ax2.set_ylabel("G'' [Pa]", fontsize=12)
ax2.set_title("Loss Modulus vs Alpha", fontsize=13)
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 6. NLSQ Fitting

In [ ]:
# Fit to SAOS data (complex modulus)
# Combine G' and G'' into complex array for fitting
G_star_data = G_prime_data + 1j * G_double_prime_data

model_fit = FIKH(include_thermal=False, alpha_structure=0.5)

# Use fewer cycles for faster fitting
t0 = time.time()
model_fit.fit(omega_data, np.abs(G_star_data), test_mode="oscillation", 
              gamma_0=0.001, n_cycles=3, method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model_fit.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Plot SAOS fit with uncertainty band
fig, ax = plot_nlsq_fit(
    omega_data, np.abs(G_star_data), model_fit, test_mode="oscillation",
    param_names=param_names, log_scale=True,
    xlabel="Frequency [rad/s]", ylabel="|G*| [Pa]",
    title="FIKH SAOS Fit",
    gamma_0=0.001, n_cycles=3,
)

# Overlay G' and G'' data points
ax.loglog(omega_data, G_prime_data, "s", color="C2", markersize=6, label="G' data", zorder=4)
ax.loglog(omega_data, G_double_prime_data, "^", color="C3", markersize=6, label="G'' data", zorder=4)
ax.legend(fontsize=9)

display(fig)
plt.close(fig)

## 7. Bayesian Inference (Optional - Slow)

**Note**: SAOS Bayesian inference is computationally expensive because each likelihood evaluation requires running time-domain simulations at every frequency. The cells below are optional and may take 10+ minutes to complete. For practical workflows, consider:

1. Using NLSQ for quick parameter estimation (as shown above)
2. Running Bayesian inference overnight or on a cluster
3. Using fewer frequency points (5-10 instead of 40)

In [ ]:
# Bayesian inference (OPTIONAL - may take 10+ minutes)
# Skip this cell for quick demos; uncomment to run full Bayesian inference

SKIP_BAYESIAN = True  # Set to False to run Bayesian inference

if not SKIP_BAYESIAN:
    initial_values = {name: model_fit.parameters.get_value(name) for name in param_names}
    
    NUM_WARMUP = 100
    NUM_SAMPLES = 200
    NUM_CHAINS = 1
    
    print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
    t0 = time.time()
    result = model_fit.fit_bayesian(
        omega_data,
        np.abs(G_star_data),
        test_mode="oscillation",
        gamma_0=0.001,
        n_cycles=3,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        initial_values=initial_values,
        seed=42,
    )
    t_bayes = time.time() - t0
    print(f"\nBayesian inference time: {t_bayes:.1f} s")
else:
    print("Skipping Bayesian inference (SKIP_BAYESIAN=True)")
    print("Set SKIP_BAYESIAN=False to run full inference (may take 10+ minutes)")
    result = None

In [ ]:
# Convergence and ArviZ diagnostics (only if Bayesian was run)
if result is not None:
    all_pass = print_convergence_summary(result, param_names)
    
    print("\n### Convergence Diagnostics")
    print("| Metric | Target | Interpretation |")
    print("|--------|--------|----------------|")
    print("| **R-hat** | < 1.01 | Chain convergence |")
    print("| **ESS** | > 400 | Effective samples |")
    print("| **Divergences** | < 1% | Sampling quality |")
    
    display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)
else:
    print("Bayesian inference was skipped. No convergence diagnostics available.")

### Convergence Diagnostics

**Diagnostic Targets:**

| Metric | Target | Interpretation |
|--------|--------|----------------|
| **R-hat** | < 1.01 | Chain convergence |
| **ESS** | > 400 | Effective samples |
| **Divergences** | < 1% | Sampling quality |

In [ ]:
# Alpha posterior (only if Bayesian was run)
if result is not None:
    posterior = result.posterior_samples
    alpha_samples = posterior["alpha_structure"]
    alpha_median = np.median(alpha_samples)
    alpha_lo, alpha_hi = np.percentile(alpha_samples, [2.5, 97.5])
    true_alpha = model.parameters.get_value("alpha_structure")
    
    print("\nFractional Order Recovery:")
    print("=" * 50)
    print(f"  True alpha:      {true_alpha:.3f}")
    print(f"  Posterior:       {alpha_median:.3f} [{alpha_lo:.3f}, {alpha_hi:.3f}]")
else:
    print("Bayesian inference was skipped. No posterior analysis available.")

In [ ]:
# Parameter comparison (only if Bayesian was run)
if result is not None:
    posterior = result.posterior_samples
    print_parameter_comparison(model_fit, posterior, param_names)
else:
    print("Bayesian inference was skipped. Showing NLSQ parameters only:")
    for name in param_names:
        val = model_fit.parameters.get_value(name)
        print(f"  {name:15s} = {val:.4g}")

## 8. Save Results

In [ ]:
if result is not None:
    save_fikh_results(model_fit, result, "fikh", "saos", param_names)
    print("\nResults saved.")
else:
    print("Bayesian results not saved (inference was skipped).")

## Key Takeaways

1. **SAOS reveals fractional viscoelasticity** via Cole-Cole depression
2. **Depression angle $\theta = (1-\alpha)\pi/2$ directly measures $\alpha$**
3. **Power-law regimes** $G', G'' \sim \omega^{\alpha}$ at low/high frequency
4. **Fractional Maxwell modulus** generalizes classical single-mode response
5. **Frequency-domain data provides complementary constraint** on $\alpha$
6. **Residual analysis** of moduli confirms fractional vs classical behavior

---

## Further Reading

- **[FIKH SAOS Protocol](../../docs/source/models/fikh/fikh.rst#saos-small-amplitude-oscillatory)**: Fractional Maxwell derivation and power-law asymptotics
- **[Cole-Cole Analysis](../../docs/source/models/fikh/fikh.rst#cole-cole-signature)**: Depression angle interpretation and experimental fitting

### Key References

1. Jaishankar, A. & McKinley, G.H. (2014). "A fractional K-BKZ constitutive formulation." *J. Rheol.*, 58, 1751-1788.
2. Mainardi, F. (2010). *Fractional Calculus and Waves in Linear Viscoelasticity*. Imperial College Press.

### Next Steps

**Next**: NB06 (LAOS) — nonlinear oscillatory response with intra-cycle memory effects